In [5]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

device="cuda" if torch.cuda.is_available() else "cpu"

model_name="distilbert/distilgpt2"
# model_name= "google-t5/t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
encoded_input = tokenizer("Hugging Face is an open-source company", return_tensors="pt").to(device)


model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16).to(device)


eos = tokenizer(tokenizer.eos_token, return_tensors="pt", return_token_type_ids=False).input_ids.flatten()[0].item()
eos = torch.tensor([eos]).to(device).reshape(1,1)






In [10]:
def exact_mode_algo(model, encoded_input, eos:int):
    def get_next_log_probs(y, model):
        with torch.no_grad():
            outputs = model(input_ids=y)
            logits = outputs.logits
        
        next_token_logits = logits[:, -1, :]
        return torch.nn.functional.log_softmax(next_token_logits, dim=-1)[0]

    #y is the prompt with additions, p is the probability of y, gamma is the current best probability, eos is the eos string
    def DFS(  y:str, p:float, gamma:float,model, eos=1, depth=0, max_depth=-1):

        #If we reached max depth without finishing
        if(depth>max_depth):
            return (y,gamma*2)

        #if y is finished, return the node
        if(y[0,-1]==eos):
            print(f"p is {p}, y is {y}, gamma is {gamma}", flush=True)
            return (y,p)
        
        #else, search for the best way to complete y
        
        #exclude the pad token
        log_probs=get_next_log_probs(y, model)

        arange=torch.arange(len(log_probs)).to(log_probs.device)
        best_y=y
        for idx, log_prob in enumerate(log_probs):
            newP = p + log_prob 
            #if we're doing better than the best one so far
            if newP > gamma:
                #do a DFS
                appended_y=torch.concat((y, arange[idx].reshape(1,1)), axis=1)
                new_y, new_gamma = DFS( y=appended_y, p=newP, gamma=gamma, model=model, eos=eos, depth=depth+1, max_depth=max_depth)
                if new_gamma > gamma:
                    best_y=new_y
                    gamma=new_gamma

        return best_y, gamma

    y=encoded_input.input_ids
    ended_y=torch.concat((y, eos), axis=1)
    start_gamma=get_next_log_probs(y=ended_y, model=model)[eos]
    best_y, gamma = DFS(y=y, gamma=start_gamma,p=0, model= model, eos=eos, depth=0, max_depth=100)
    return best_y, gamma



start_gamma is tensor([[-12.9688]], dtype=torch.float16)
p is -12.375, y is tensor([[48098,  2667, 15399,   318,   281,  1280,    12, 10459,  1664,     0,
         50256]]), gamma is tensor([[-12.9688]], dtype=torch.float16)


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x1051ef580>>
Traceback (most recent call last):
  File "/opt/miniconda3/envs/crfm-helm2/lib/python3.9/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


KeyboardInterrupt: 